In [ ]:
from analysis.analysis import *
import matplotlib.pyplot as plt
import os

root = "defaults"
gid = 108
default_LC_gid = 10

Read the raw sim data for a grid ID

In [ ]:
data_root = f"/mnt/c/Users/pedro/OneDrive - Johns Hopkins/Documents/Research/hopper/_server/sim_data/{root}"

grid_df = read_fulltake(
    os.path.join(data_root, f"parquets/fulltake_gid{gid}.parquet"), scale_position=True
)

grid_df.rid.unique().size, yaml.safe_load(
    open(os.path.join(data_root, f"configs/grid_id{gid}/cell.yaml"), "rb")
), yaml.safe_load(
    open(os.path.join(data_root, f"configs/grid_id{gid}/simbox.yaml"), "rb")
)

Histogram of hopping times _vs._ default limit-cycle cell

In [ ]:
try:
    plt.figure(figsize=(3, 2), dpi=100)
    plt.hist(
        np.load(f"../_server/sim_data/{root}/hopping times/fulltake_gid{gid}.npy"),
        density=True,
        bins=40,
        alpha=0.75,
        color="cornflowerblue",
        label=f"Grid {gid}",
    )
    plt.hist(
        np.load(f"../_server/sim_data/{root}/hopping times/fulltake_gid{10}.npy"),
        density=True,
        bins=40,
        alpha=0.75,
        color="salmon",
        label=f"Grid {default_LC_gid}",
    )
    plt.xlabel("Hopping times (hr)")
    plt.legend()
    plt.show()
except FileNotFoundError:
    print("Make sure to first run compute_hopping_times.py")

Apply a time filter (min) to raw data and compute $x, v, a$
- Sims collect at a rate of 3 min

In [ ]:
grid_df_samples = apply_time_filter(grid_df, dt=3, base_rate=3)

nbins = 32
xva_df = get_xva_df(
    grid_df_samples,
    nbins,
    yfile=os.path.join(data_root, f"configs/grid_id{gid}/simbox.yaml"),
)

In [ ]:
arr = np.ones(shape=(nbins, nbins)) * np.nan
indx = np.array(n.index.tolist())[:, ::-1]
arr[indx[:, 0], indx[:, 1]] = n.values

In [ ]:
plt.figure(figsize=(3, 3))
plt.imshow(np.log(arr), extent=[0, 50, 0, 50], origin="lower")
plt.colorbar()

Plot of $F(x, v)$ with streamlines and $\sigma(x, v)$

In [ ]:
def dynamicsplot(ax, F, bounds, labels, end_pts, delta=1, **kwargs):
    X, Y, init_pts = full_lattice(F, *bounds)

    # raw
    v, f = evaluate_F_v_x0(Y, F, bounds[-1], delta=delta)
    lim = np.fabs([v.min(), v.max()]).min()
    v_interp = np.linspace(-lim, lim, 30)
    f_interp = np.interp(v_interp, v, f)

    kwargs = {"lw": 2, "color": "black"} | kwargs
    ax.plot(v_interp, f_interp, "-o", **kwargs)
    ax.grid(which="major")
    ax.set_xlabel(r"$v$ ($\mu$m/hr)")
    ax.set_ylabel(r"$F(x\rightarrow 0, v)$ ($\mu$m/hr$^2$)")


bounds, F, sigma = compute_F_sigma(xva_df, nbins=nbins, min_pts=2)
xmin, xmax = bounds["x"]
vmin, vmax = bounds["v"]
bounds = (xmin, xmax, vmin, vmax, nbins)

dynamicsplot(plt.gca(), F, bounds, "", "")

In [ ]:
bounds, F, sigma = compute_F_sigma(xva_df, nbins=nbins, min_pts=2)

xmin, xmax = bounds["x"]
vmin, vmax = bounds["v"]
x_c = (xmax + xmin) / 2
xmin, xmax = np.array([xmin, xmax]) - x_c  # center along x
bounds_tup = (xmin, xmax, vmin, vmax, nbins)

plot_title = make_title(xva_df)
title = {"label": plot_title, "fontsize": 20}

# init_pts[:, 1] += 20
fig, ax = plt.subplots(1, 2, figsize=(11, 3), dpi=300)

X, Y, init_pts = full_lattice(F, xmin, xmax, vmin, vmax, nbins)
init_pts = linear_lattice(xmin, xmax, vmin, vmax, n_pts=100, s=1)

ax[0].set_title(**title)
ax[0].axhline(y=0, lw=1, color="black", zorder=2)

F_streamplot(
    F,
    bounds_tup,
    stream_init_pts=init_pts,
    imshow_kwargs={"interpolation": "bilinear", "origin": "lower", "cmap": "jet"},
    streamplot_kwargs={
        "integration_direction": "forward",
        "color": "black",
        "broken_streamlines": False,
        "density": 1,
        "linewidth": 0.4,
    },
    do_try=True,
    vector_field=False,
    ax=ax[0],
)

if gid == -1:
    import pickle

    data = pickle.load(open(f"maps/map_{gid}.pkl", "rb"))
    end_pts = data["end_pts"]
    ax[0].scatter(
        end_pts[:, 0] - x_c,
        end_pts[:, 1],
        s=50,
        color="red",
        marker="x",
        zorder=10,
    )

ax[1].remove()
ax[0].set_title("")

plt.subplots_adjust(wspace=0.5)
plt.show()

In [ ]:
_df = grid_df[grid_df.rid == np.random.randint(grid_df.rid.unique().size)]
_df.plot(x="time[hr]", y="x", color="black", legend=False)
plt.gca().hlines([133, 167], *plt.gca().get_xlim()), _df.iloc[0].seed

In [ ]:
plt.plot(_df.x)

Phase separation for bistable cases

In [ ]:
from importlib import reload
from analysis import analysis

reload(analysis)
from analysis.analysis import lattice_to_image

In [ ]:
X, Y, init_pts = full_lattice(F, xmin, xmax, vmin, vmax, nbins)

labels, end_pts = get_labels(init_pts, X, Y, F, x_L=135, x_R=165)
end_pts = np.array(end_pts)
set(labels)

In [ ]:
# labels_ = np.where(np.array(labels) == "wheat", "gainsboro", labels)
img = lattice_to_image(init_pts, labels, bounds_tup)


arr = np.where(img == 1, 0, 1)
X = get_separatrices(img, levels=[1.5], origin="lower", extent=[xmin, xmax, vmin, vmax])

arr = np.where(img == 1, 1, 0)
X2 = get_separatrices(
    arr, levels=[0.5], origin="lower", extent=[xmin, xmax, vmin, vmax]
)

# plot
plt.figure(figsize=(3, 3), dpi=300)
plt.vlines(
    x=135, ymin=vmin, ymax=vmax, linewidth=1, colors=["black"], linestyles=["dashed"]
)
plt.vlines(
    x=165, ymin=vmin, ymax=vmax, linewidth=1, colors=["black"], linestyles=["dashed"]
)

plt.scatter(init_pts[:, 0], init_pts[:, 1], color=labels, s=5)
# plt.plot(X[:, 0], X[:, 1], lw=1.5, color="black")
plt.plot(X2[:, 0], X2[:, 1], lw=1.5, color="black")

# plt.scatter(end_pts[:, 0], end_pts[:, 1], marker="*", s=10, color="black")
plt.xlabel(r"$x$ ($\mu$m)")
plt.ylabel(r"$v$ ($\mu$m/hr)")

xlim = plt.xlim()
ylim = plt.ylim()

# Add fancy legends
import matplotlib.patches as mpatches

legend_dict = {
    "salmon": mpatches.Patch(color="salmon", label="Left"),
    "cornflowerblue": mpatches.Patch(color="cornflowerblue", label="Right"),
    "wheat": mpatches.Patch(color="wheat", label="Middle"),
    "red": mpatches.Patch(color="red", label="Unkn"),
}
legend = []
for c in np.unique(labels):
    legend.append(legend_dict[c]) if c != "gainsboro" else None
plt.legend(handles=legend, bbox_to_anchor=(1.01, 1))
plt.show()

In [ ]:
sys.exit(01)

In [ ]:
def process_gid(gid):
    import pickle

    data = pickle.load(open(f"maps/map_{gid}.pkl", "rb"))
    if len(data["labels"]) > 0:
        return data["F"], data["bounds"], data["labels"], data["end_pts"]
    return data["F"], data["bounds"]

In [ ]:
F, bounds, labels, end_pts = process_gid(110)
xmin, xmax, vmin, vmax, nbins = bounds

np.vstack(np.meshgrid(np.arange(nbins), np.arange(nbins)))

In [ ]:
dx = (xmax - xmin) / nbins
dv = (vmax - vmin) / nbins
X, Y = np.meshgrid(np.arange(nbins), np.arange(nbins))
X = X * dx + xmin + dx / 2
Y = Y * dv + vmin + dv / 2
init_pts = np.hstack([X.reshape(-1, 1), Y.reshape(-1, 1)])

labels, end_pts = get_labels(init_pts, X, Y, F, x_L=135, x_R=165)

In [ ]:
img = lattice_to_image(init_pts, labels, bounds)

arr = np.where(img == 1, 0, 1)
X = get_separatrices(arr, levels=[0.5], origin="lower", extent=[xmin, xmax, vmin, vmax])

arr = np.where(img == 1, 1, 0)
X2 = get_separatrices(
    arr, levels=[0.01], origin="lower", extent=[xmin, xmax, vmin, vmax]
)

# plot
plt.figure(figsize=(3, 3), dpi=300)
plt.vlines(
    x=135, ymin=vmin, ymax=vmax, linewidth=1, colors=["black"], linestyles=["dashed"]
)
plt.vlines(
    x=165, ymin=vmin, ymax=vmax, linewidth=1, colors=["black"], linestyles=["dashed"]
)

plt.scatter(init_pts[:, 0], init_pts[:, 1], color=labels, s=5)
# plt.plot(X[:, 0], X[:, 1], lw=2, color="black")
plt.plot(X2[:, 0], X2[:, 1], lw=2, color="black")

# plt.scatter(end_pts[:, 0], end_pts[:, 1], marker="*", s=10, color="black")
plt.xlabel(r"$x$ ($\mu$m)")
plt.ylabel(r"$v$ ($\mu$m/hr)")

xlim = plt.xlim()
ylim = plt.ylim()

# Add fancy legends
import matplotlib.patches as mpatches

legend_dict = {
    "salmon": mpatches.Patch(color="salmon", label="Left"),
    "cornflowerblue": mpatches.Patch(color="cornflowerblue", label="Right"),
    "wheat": mpatches.Patch(color="wheat", label="Middle"),
    "red": mpatches.Patch(color="red", label="Unkn"),
}
legend = []
for c in np.unique(labels):
    legend.append(legend_dict[c]) if c != "gainsboro" else None
plt.legend(handles=legend, bbox_to_anchor=(1.01, 1))
# plt.xlim((135, 165))
# plt.ylim((-20, 20))
plt.show()

In [ ]:
class IntegrateF:
    def __init__(self, X, V, F):
        self.X = X
        self.V = V
        self.F = F

    def integrate(self, x0, v0, T, seed=0):
        x = [x0]
        v = [v0]
        delta_t = 0.00075 * 8 / 60  # hr
        delta_t = 0.01

        rng = np.random.default_rng(seed=seed)

        for t in range(1, T):
            if np.isnan([x[t - 1], v[t - 1]]).any() or not (
                self.X.min() <= x[t - 1] <= self.X.max()
                and self.V.min() <= v[t - 1] <= self.V.max()
            ):
                break

            i, j = self.get_bin_index(x[t - 1], v[t - 1], self.X, self.V)
            _x = x[t - 1] + v[t - 1] * delta_t
            _v = v[t - 1] + self.F[i, j] * delta_t
            x.append(_x)
            v.append(_v)
        return np.vstack([x, v]).T

    @staticmethod
    def get_bin_index(x, v, X, V):
        return np.searchsorted(V, v, side="left"), np.searchsorted(X, x, side="left")


F, bounds, labels, end_pts = process_gid(10)
xmin, xmax, vmin, vmax, nbins = bounds
X, V, init_pts = full_lattice(F, *bounds)
X = X[0]
V = V[:, 0]

integrator = IntegrateF(X, V, F)
k = 30
x_traj = integrator.integrate(*init_pts[k], T=500)

plt.imshow(F, origin="lower", extent=[xmin, xmax, vmin, vmax])
plt.scatter(*init_pts[k], s=40, color="red")
plt.scatter(x_traj[:, 0], x_traj[:, 1], s=10, color="cyan")
plt.axis("auto")

In [ ]:
import sys

sys.exit(1)